In [ ]:
# Parameters (do not containerize this cell)
param_data_filename <- "Template_MBO_Example_raw.xlsx"
param_metadata_sheet <- "METADATA"
param_data_sheet <- "BIRDS"
param_user_email = "[the e-mail address you used to log into NaaVRE]"
param_use_dummy_data <- 1
conf_temporary_data_directory <- "/tmp/data"
conf_virtual_lab_biotisan_euromarec <- "vl-biotisan-euromarec"
conf_minio_endpoint <- "scruffy.lab.uvalight.net:9000"
conf_minio_region <- "nl-uvalight"
conf_minio_public_bucket <- "naa-vre-public"
conf_minio_user_bucket <- "naa-vre-user-data"

In [ ]:
# Secrets (do not containerize this cell)
library("SecretsProvider")

secretsProvider <- SecretsProvider()
secret_minio_access_key = ""
secret_minio_access_key = secretsProvider$get_secret("secret_minio_access_key")
secret_minio_secret_key = ""
secret_minio_secret_key = secretsProvider$get_secret("secret_minio_secret_key")

In [ ]:
# MinIO data retriever
library("readxl")
library("aws.s3")

Sys.setenv("AWS_S3_ENDPOINT" = conf_minio_endpoint,
           "AWS_DEFAULT_REGION" = conf_minio_region,
           "AWS_ACCESS_KEY_ID" = secret_minio_access_key,
           "AWS_SECRET_ACCESS_KEY" = secret_minio_secret_key)

# Download file from bucket S3
if (param_use_dummy_data) {
        file_path <- paste(conf_virtual_lab_biotisan_euromarec, param_data_filename, sep="/")
        print(sprintf("Using dummy data for testing purposes. Set param_use_dummy_data to 0 to use your own data. Downloading data from %s / %s", conf_minio_public_bucket, file_path))
        aws.s3::save_object(bucket=conf_minio_public_bucket, object=file_path, file=param_data_filename)
    } else {
        file_path <- paste(param_user_email, param_data_filename, sep="/")
        print(sprintf("Downloading data from %s / %s", conf_minio_user_bucket, file_path))
        aws.s3::save_object(bucket=conf_minio_user_bucket, object=file_path, file=param_data_filename)
}

# Load data & metadata
metadata <- read_excel(param_data_filename, sheet = param_metadata_sheet) #Load metadata sheet
data <- read_excel(param_data_filename, sheet = param_data_sheet) #Load data sheet

# Ensure the temporary data storage directory exists
dir.create(conf_temporary_data_directory, showWarnings = FALSE)

# Write (meta)data to files
metadata_as_csv_filename <- "metadata.csv"
data_as_csv_filename <- "data.csv"
metadata_from_excel_path <- paste(conf_temporary_data_directory, metadata_as_csv_filename, sep="/")
data_from_excel_path <- paste(conf_temporary_data_directory, data_as_csv_filename, sep="/")
print(sprintf("Storing metadata in: %s, and data in %s", metadata_from_excel_path, data_from_excel_path))
write.csv(metadata, file = metadata_from_excel_path)
write.csv(data, file =  data_from_excel_path)